In [ ]:
pip install transformers datasets torch gradio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transcripts_dir = '/content/drive/My Drive/Colab Notebooks/Project Transcrips'
model_dir = '/content/drive/My Drive/Colab Notebooks/Project Model'

In [ ]:
# Load raw transcripts from the file
with open('/content/drive/My Drive/transcripts_separated-1.txt', 'r', encoding='utf-8') as file:
    raw_transcripts = file.read().split("="*50)  # Assuming the separator is ========


In [ ]:
import re

def clean_text(text):
    # Remove non-verbal cues like [Applause], [Laughter], etc.
    text = re.sub(r"\[.*?\]", "", text)
    # Replace multiple spaces with a single space
    text = re.sub(r"\s+", " ", text)
    # Strip leading and trailing spaces
    text = text.strip()

    # Insert commas for likely sentence continuations
    text = re.sub(r"\b(and|but|so|or)\b", r", \1", text, flags=re.IGNORECASE)

    # Insert periods before capital letters indicating new sentences
    text = re.sub(r"(?<!\w)([a-zA-Z]+)(\s+)([A-Z])", r"\1.\2\3", text)

    # Capitalize the start of each sentence after periods
    sentences = text.split(". ")
    sentences = [sentence.capitalize() for sentence in sentences]
    text = ". ".join(sentences)

    # Add a period at the end if missing
    if text and text[-1] not in ".!?":
        text += "."

    return text

# Clean all transcripts
cleaned_transcripts = [clean_text(transcript) for transcript in raw_transcripts]


In [ ]:
# Save cleaned transcripts as a text file
cleaned_file_path = '/content/drive/MyDrive/Colab Notebooks/cleaned_transcripts.txt'
with open(cleaned_file_path, 'w', encoding='utf-8') as file:
    for transcript in cleaned_transcripts:
        file.write(transcript + '\n\n')

print(f"Cleaned transcripts saved at {cleaned_file_path}")


Cleaned transcripts saved at /content/drive/MyDrive/Colab Notebooks/cleaned_transcripts.txt


In [ ]:
pip install datasets

In [ ]:
from datasets import Dataset

# Create dataset from cleaned transcripts
dataset = Dataset.from_dict({'text': cleaned_transcripts})

# Split into training and testing sets
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


In [ ]:
from datasets import Dataset
from transformers import GPT2Tokenizer

# Assume `cleaned_transcripts` contains the cleaned text data
# Create a dataset from the cleaned transcripts
dataset = Dataset.from_dict({'text': cleaned_transcripts})

# Split into training and testing sets
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Load the GPT-2 tokenizer and set up padding
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set the pad token to EOS token

# Tokenize datasets
def tokenize(batch):
    encoding = tokenizer(batch["text"], padding=True, truncation=True, max_length=512)
    encoding["labels"] = encoding["input_ids"].copy()  # Use input_ids as labels for language modeling
    return encoding

# Apply tokenization to both training and testing datasets
train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=["text"])
test_dataset = test_dataset.map(tokenize, batched=True, remove_columns=["text"])

# Set format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/427 [00:00<?, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [ ]:
pip install --upgrade sympy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 44.9 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable WandB

from transformers import GPT2LMHeadModel, Trainer, TrainingArguments


In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/ComBot",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('/content/drive/MyDrive/ComBot')
tokenizer.save_pretrained('/content/drive/MyDrive/ComBot')

print("Fine-tuned model saved successfully!")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,No log,3.005940


Epoch,Training Loss,Validation Loss
1,No log,3.005940
2,No log,2.944750
3,3.141500,2.925697


Fine-tuned model saved successfully!


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/ComBot')
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/ComBot')

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=150,
                             num_return_sequences=1, do_sample=True, top_k=50, top_p=0.95)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs="text",
    title="My Fine-Tuned Chatbot",
    description="Interact with your chatbot trained on your custom transcripts!"
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0195e503b9982edcb0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
